# Splashtop Secure Workspace Automation

Splashtop Secure Workspace provides a rich set of APIs with full automation capability for managing and working within the workspace. The secure workspace offers IT administrators and end users an all-in-one platform to access their applications and data, making it crucial to automate the provisioning of the workspace and its applications. The automation capability empowers IT admins, developers, and end users with unprecedented flexibility to manage and work within the workspace. Some examples include:

- IT admins can automate the provisioning of the workspace and applications for end users.
- Developers can automate and orchestrate their geo-distributed applications using a single piece of code.
- End users can automate their daily tasks within the workspace.
- System admins can automate the network configuration within the workspace.

## Secure Workspace as Code (SWaC)

With the full automation capability, this notebook can fulfill the following example use cases without the need for manual installation of any applications or configuration/setup:

- Authenticate to your workspace.
- Manage the workspace as an admin, for example, provisioning a sample RESTful API application for all user groups.
- Tunnel to the remote RESTful API application and invoke the API.



##Prerequisites:
    - Splashtop Secure Workspace account
    - Python and Jupyter Notebook installed

## Generate Secure Workspace API key:
    - Log in to the Splashtop Business web console.
    - Go to Settings -> API Keys -> Generate API Key.
    - Copy the API key.
    - Update the `apikey.py` file with the API key.


## Install Python Libraries

In [ ]:

!pip install requests PyJWT

## Setup Variables
setup the following cell with variables for your environment

In [122]:
import os
import json
import subprocess
import requests
import time
from apikey import apikey

DEBUG = True
curr_path = os.getcwd()

# Variables - fill these in
# apikey from import
server_url = 'demo.stage.ztw.splashtop.com'
org_name = 'demo'
env = 'stage'
site_name = 'cloud-connector'
network_name = 'demo'
edge_name = 'US-WEST-1'
EXE_FILE = ''
ERROR_MESSAGE = ''

## Authentication and Access Tokens

Run the following cell to authenticate to the Secure Workspace and obtain authentication tokens. There are two types of tokens available:

* `iam_token`: Used for Secure Workspace IAM operations, such as authentication, user management, group management, etc.
* `sdp_token`: Used for Secure Workspace zero trust application management operations, such as application provisioning, client management, etc.

In [ ]:
# Get SDP token
apikey_token = {"apikey": apikey}
response = requests.post(f"https://{server_url}/iam/v1/apikey/authenticate", json=apikey_token)
if response.status_code == 200 and response.json().get('code')==0:
        sdp_token_new = response.json().get('data').get('sdp_token')
        iam_token_new = response.json().get('data').get('iam_token')
        if DEBUG:
            print(f'sdp_iam_token.json: {response.json()}')
        if not sdp_token_new:
            ERROR_MESSAGE += "Error: sdp_token_new is empty, get sdp token failed \n"
else:
        ERROR_MESSAGE += f"Error: HTTP {response.status_code} - {response.text} \n"




## Retrieve Organization Information from IAM Token

By using the IAM token, we can retrieve the organization information. The organization represents the top-level container of the Secure Workspace. All users, groups, applications, and other entities reside within the organization. It provides a multi-tenant capability for each workspace account. The organization ID is utilized for all Secure Workspace API calls.


In [125]:
import jwt
import json

# Base64 encoded JSON token
encoded_token = iam_token_new

# Decoding the JWT
decoded_token = jwt.decode(encoded_token, options={"verify_signature":False})

# Loading the payload into a JSON object
payload_json = json.loads(json.dumps(decoded_token))

# Extracting the organization ID
organization_id = payload_json['current_organization_id']


## Retrieve Group Information
By utilizing the organization ID, we can retrieve all the groups that exist within the organization. In this example, we will obtain information about a built-in group called "All User".

In [ ]:
# Get group_id for all user group
headers = {"Accept": "application/json, text/plain, */*", "Authorization": f"Bearer {iam_token_new}", "Content-Type": "application/json;charset=UTF-8"}
response = requests.get(f"https://{server_url}/iam/v1/organization/{organization_id}/groups?page_size=99999", headers=headers)

group_id  = ''
if response.status_code == 200 and response.json().get('code')==0:
        # find the "All User" group_id
        group_id = response.json().get('data').get('list')[0].get('group_id')
elif response.status_code != 200:
        ERROR_MESSAGE += f"Error: HTTP {response.status_code} - {response.text} \n"
        print (ERROR_MESSAGE)
else:
        ERROR_MESSAGE += f"Error: {response.json().get('message')} \n" 
        print (ERROR_MESSAGE)       
print(f'group_id : {group_id }')


# Retrieve Application Information & Provision Application to the "All User" Group

By using the organization ID, we can retrieve all the applications within the organization. In this example, we will obtain information about a sample application called "ztw-restful-cloud". Subsequently, we will provision the application to the "All User" group.


In [ ]:
# find the ztw-restful-cloud application
headers = {"Accept": "application/json, text/plain, */*", "Authorization": f"Bearer {sdp_token_new}", "Content-Type": "application/json;charset=UTF-8"}
response = requests.get(f"https://{server_url}/ztna/v1/application?search=&order=&desc=false&org_id={organization_id}", headers=headers)

application = {} 
if response.status_code == 200 and response.json().get('code')==0:
        # find the "All User" group_id
        apps = response.json().get('data').get('list')
        for app in apps:
            if app.get('name') == 'ztw-restful-cloud':
                application = app
                break 
elif response.status_code != 200:
        ERROR_MESSAGE += f"Error: HTTP {response.status_code} - {response.text} \n"
        print (ERROR_MESSAGE)
else:
        ERROR_MESSAGE += f"Error: {response.json().get('message')} \n" 
        print (ERROR_MESSAGE)       
print(f'application: {application}')
application_id = application.get('id')

#update the application by assigning all users group to it

# Check if the application not have the group_id, append all users group to it
if group_id not in application.get('group_ids'):
        application.get('group_ids').append(group_id)


headers = {"Accept": "application/json, text/plain, */*", "Authorization": f"Bearer {sdp_token_new}", "Content-Type": "application/json;charset=UTF-8"}
response = requests.put(f"https://{server_url}/ztna/v1/application/{application_id}",  headers=headers, data=json.dumps(application))

if response.status_code == 200 and response.json().get('code')==0:
        print (f"Update Application Success: {response.json().get('message')} \n")
elif response.status_code != 200:
        ERROR_MESSAGE += f"Error: HTTP {response.status_code} - {response.text} \n"
        print (ERROR_MESSAGE)
else:
        ERROR_MESSAGE += f"Error: {response.json().get('message')} \n" 
        print (ERROR_MESSAGE)       



## Automate Installation of Secure Workspace CLI

To establish connections with secure workspace remote applications from automation code, it is necessary to install the secure workspace CLI. The provided code automates the installation process using brew, assuming that you are using macOS.


In [ ]:
#on MacOS install brew if not installed
! /bin/bash -c '$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/HEAD/install.sh)'

In [131]:
#on MacOS brew add tap if not added
!brew tap SplashtopInc/workspace


In [ ]:
#on MacOS brew install the secure workspace cli client
!brew install sdpc

## Automate Network Tunneling to the Remote Application

Once the secure workspace CLI is installed, we can automate the network tunneling to the remote application. In this example, we will establish a tunnel to the remote RESTful API application and invoke the API. 

When running this cell, please note that the notebook will prompt for your local operating system password in order to run `sudo`. You should then wait for the CLI to establish the tunnel. If the connection is successfully established, you will see the following output:

```bash
ztw-restful-cloud connect success


In [ ]:
#use sdpc to connect to the ztw-restful-cloud application

# generate the network connection script
from getpass import getpass
import subprocess

password = getpass()
script_content = f"""
#!/bin/bash
echo {password} | sudo -S /opt/homebrew/bin/sdpc -apikey {apikey} -sdp_server https://stage.ztw.splashtop.com -org demo  {application_id}
"""

# Write the script_content to a file
with open('script.sh', 'w') as f:
    f.write(script_content)

# Change the permission of the script to be executable
!chmod +x script.sh

# Run the script in a subprocess, if the connection established, you should see the output: ztw-restful-cloud connect success 
subprocess.Popen(['./script.sh'],  shell=True)


## API Call to the Secure Workspace Managed Remote Application

Now that the network tunneling has been established to the remote RESTful application, we can make API calls to it from this notebook. The example application is a simple RESTful API that returns the following JSON response:

```json
{
    "message": "Hello World!"
}


In [ ]:
headers = {"Accept": "application/json, text/plain, */*", "Authorization": f"Bearer {sdp_token_new}", "Content-Type": "application/json;charset=UTF-8"}
response = requests.get(f"http://ztenv-myrestful.default.svc.cluster.local:8080/", headers=headers)

if response.status_code == 200 :
    print (f"Success: {response.json()} \n")
    
else: 
    ERROR_MESSAGE += f"Error: HTTP {response.status_code} - {response.text} \n"
    print (ERROR_MESSAGE)


## Clean Up and Terminate the Tunnel

In this cell, we will clean up the application and terminate the tunnel. The script will prompt for your local operating system password in order to run `sudo`. After entering your password, please wait for the CLI to terminate the tunnel.


In [ ]:
# kill the process to disconnect the secure workspace
!echo {getpass()} | sudo -S killall sdpc
!rm -f script.sh